# Import Libraries

In [1]:
# Standard libraries
import json
import numpy as np
import pandas as pd
import scipy.signal as signal

# Custom libraries
from Functions import processing

# Import Epoched Data and Settings

In [ ]:
# Load list of files to import
files = [  
    "sub-P003_ses-S001_task-T1_run-001_eeg"
]

# Get unique subject IDs
subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate variables to store EEG data and settings
eeg_epochs = [None] * len(files)
settings = [None] * len(files)

# Import data
for f, file in enumerate(files):
    # Import EEG data, since it is stored in a compressed numpy file (.npz) we need to use the np.load function 
    loaded_data = np.load(f"Data\\Pilot2\\EEG\\sub-P003\\ses-S001\\eeg\\{file}.npz", allow_pickle=True)

    # Access the data for each stimulus
    eeg_epochs[f] = {stim_label: loaded_data[stim_label] for stim_label in loaded_data.files}

    # Import settings
    with open(f"Data\\Pilot2\\EEG\\sub-P003\\ses-S001\\eeg\\{file}.json", "r") as file_object:
        settings[f] = json.load(file_object)

# Calculate PSD of all Epochs

In [3]:
# PSD settings
window_size = 5  # 10 = 0.1 Hz resolution, 5 = 0.2 Hz resolution, 2 = 0.5 Hz resolution

# Preallocate variables
eeg_f = [None] * len(files)
eeg_pxx = [None] * len(files)  # Preallocate to list in case not all files have the same number of channels

# Compute PSD for each file
for f, file in enumerate(files):
    eeg_f[f] = {}
    eeg_pxx[f] = {}

    # Compute PSD for each stimulus
    for stim_label, epochs in eeg_epochs[f].items():
        eeg_f[f][stim_label] = []
        eeg_pxx[f][stim_label] = []

        # Compute PSD for each epoch
        for epoch in epochs:
            f_values, pxx_values = signal.welch(
                x=epoch,
                fs=settings[f]["eeg_srate"],
                nfft=int(window_size * settings[f]["eeg_srate"]),
                nperseg=window_size * settings[f]["eeg_srate"],
                noverlap= (window_size * settings[f]["eeg_srate"]) * 0.5,  # 50% overlap between windows
            )
            eeg_f[f][stim_label].append(f_values)
            eeg_pxx[f][stim_label].append(pxx_values)

        # Convert lists to arrays for consistency
        eeg_f[f][stim_label] = np.array(eeg_f[f][stim_label])
        eeg_pxx[f][stim_label] = np.array(eeg_pxx[f][stim_label])

# Compute SNR for all Epochs
- SNR is calculated for each epoch and is NOT averaged per stimulus

In [4]:
# Settings
noise_band = 1    # Single-sided noise band [Hz]
nharms = 2        # Number of harmonics used
db_out = True     # Boolean to get output in dB
stim_freq = 10.0  # Example frequency, replace with your actual frequency

snr = [None] * len(files)

for f0 in range(len(files)):
    stim_labels = list(settings[f0]["stimuli"].values())
    file_channels = settings[f0]["new_ch_names"]
    ch_idx_map = {ch: i for i, ch in enumerate(file_channels)}

    if snr[f0] is None:
        snr[f0] = {}  # Init dict per file

    for stim_idx, stim_label in settings[f0]["stimuli"].items():
        num_epochs = eeg_pxx[f0][stim_label].shape[0]
        channel_snr_list = []

        for epoch in range(num_epochs):
            snr_epoch = processing.ssvep_snr(
                f=eeg_f[f0][stim_label][epoch],  # shape: (n_freqs,)
                pxx=eeg_pxx[f0][stim_label][epoch, :, :],  # shape: (n_channels, n_freqs)
                stim_freq=stim_freq,
                noise_band=noise_band,
                nharms=nharms,
                db_out=db_out
            )
            channel_snr_list.append(snr_epoch)

        # Create array: shape = (num_epochs, total_channels)
        epoch_snr_stack = np.zeros((num_epochs, len(file_channels)))

        for epoch_idx in range(num_epochs):
            for i, ch_name in enumerate(file_channels):
                if ch_name in ch_idx_map:
                    epoch_snr_stack[epoch_idx, i] = channel_snr_list[epoch_idx][ch_idx_map[ch_name]]
                else:
                    epoch_snr_stack[epoch_idx, i] = 0  # Channel missing

        # Store per stimulus
        snr[f0][stim_label] = epoch_snr_stack  # shape: (num_epochs, total_channels)

# Export SNR

In [5]:
rows = []

for f_idx, file_snr in enumerate(snr):
    for stim_label, snr_array in file_snr.items():  # snr_array shape: (num_epochs, num_channels)
        num_epochs = snr_array.shape[0]
        for epoch_idx in range(num_epochs):
            row = {
                "file_index": f_idx,
                "stimulus": stim_label,
                "epoch": epoch_idx
            }
            for ch_idx, ch_name in enumerate(file_channels):
                row[ch_name] = snr_array[epoch_idx, ch_idx]
            rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

# Export to CSV
#df.to_csv("epochwise_snr_output.csv", index=False)